# Decision Tree 
1. 개념
    - 데이터에 있는 규칙(일련의 결정 규칙)을 학습을 통해 자동으로 찾아낸 트리 기반의 분류 규칙을 만드는 것
    - if/esle 기반으로 예측 위한 규칙을 만드는 알고리즘
    - 스무고개 게임과 비슷하다고 생각하면 된다.

2. 특징
    - 분류학습과 회귀학습 모두 적용 가능
    - 해석이 용이
    - scale에 영향을 받지 않음
    - 범주형 변수를 수치형으로 변환하지 않고 학습 가능
    - Random Forest, GBM(XGBoost, LightGBM)의 기반이 되는 알고리즘

3. 분류학습과 회귀학습
    - 분류학습
        + 범주를 가장 잘 구분할 수 있는 변수와 그 변수의 기준값을 검색
        + 위의 변수 / 기준값을 기준으로 자식 노드 구분
    - 회귀학습
        + 손실함수를 최소화하는 변수/기준값을 검색
        + 위의 변수/기준값을 기준으로 자식 노드 구분

## <라이브러리 import 및 설정>

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import graphviz
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('ggplot')
pd.set_option('max_columns', 20)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## <데이터 로드>

In [4]:
data_dir = Path('')
sub_dir = Path('')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
seed = 42

In [5]:
algo_name = 'Dtree'
feature_name = 'fe'
model_name = f'{algo_name}_{feature_name}'

sub_file = sub_dir / f'{model_name}.csv'

In [ ]:
# 데이터 불러오기
trn = pd.read_csv(trn_file, index_col=0)
tst = pd.read_csv(tst_file, index_col=0)

# 종속변수
y = trn[target_col]

# y를 제외하고 trn 생성
trn.drop(target_col, axis=1, inplace=True)
print(y.shape, trn.shape, tst.shape)
trn.head()

## <결정트리 학습>
### [파라미터]
1. min_sample_split
    - 노드를 분할하기 위한 최소한의 샘플 데이터 수
    - 디폴트는 2이고 작게 설정할수록 과적합 가능성 증가
2. min_samples_leaf
    - 말단 노드가 되기 위한 초소한의 샘플 데이터 수
    - 비대칭적 데이터의 경우는 작게 설정해야 함
3. max_leaf_nodes
    - 말단 노드의 최대 개수
4. max_features
    - 최적의 분할을 위해 고려할 최대 피처 개수
    - float형으로 쓰면 전체 피처 중 퍼센트로 고려
5. max_depth
    - 트리의 최대 깊이를 고정
    - 디폴트 = None : 완벽하게 클래스 결정 값이 될 때까지 깊이를 계속 키움
        + 과적합으로 이어질 수 있으니 적절한 값으로 제어 필요

In [8]:
# 파라미터를 default값으로 설정하고 모델 학습
DT_clf = DecisionTreeClassifier(random_state=42)
DT_clf.fit(trn, y)

DecisionTreeClassifier(random_state=42)

In [8]:
# 학습데이터의 정확도 확인
print(f'{accuracy_score(y, DT_clf.predict(trn)) * 100:.4f}%')

88.8669%


## <결정트리 시각화>
- 시각화를 위해서는 Graphviz를 내려 받은 뒤 설치해야 한다.
- Anaconda Prompt 를 ‘관리자 권한으로 실행’하여 pip install graphviz 입력하기 
- 이후 Graphviz와 파이썬 래퍼를 연결하기 위해 추가적인 환경설정 필요

In [ ]:
dot_data = export_graphviz(DT_clf, out_file=None,
                           feature_names=trn.columns,
                           filled=True,
                           rounded=True,
                           special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

## <feature_importances 확인>
- feature_importances는 ndarray 형태로 값을 반환하며 피처 순서대로 값이 할당된다.
- 값이 높을수록 해당 피처의 중요도가 높다는 의미

In [ ]:
# 피처별 정확도 확인
print('Feature importances://n{0}' .format(np.round(DT_clf.feature_importances_., 3)))

# feature importance를 column 별로 시각화
sns.barplot(x = DT_clf.feature_importances, y = trn)

## <평가 데이터 예측>
- 어떠한 전처리도 거치지 않고 기본적인 알고리즘에 그대로 적용

In [10]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [11]:
sub[target_col] = clf.predict(tst)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [12]:
sub[target_col].value_counts()

2    45299
0    29882
1     4819
Name: class, dtype: int64

## <제출파일 저장>
- 파일 제출을 위해 submission_file 저장

In [13]:
sub.to_csv(sub_file)